# Koreksi Bias Statistik

## Pendahuluan

Metode koreksi bias biasanya menggunakan fungsi transfer/koreksi yang mengubah data model menjadi data terkoreksi secara statistik. Selain Delta Method [@sperna2010; @graham2007], metode koreksi bias lain telah dikembangkan, seperti linear scalling correction, power transformation, empirical quantile mapping, dan parametric quantile mapping [@delei2019]. Beberapa keterbatasan dalam melakukan koreksi bias, yaitu umumnya tidak memiliki dasar fisika yang kuat, menghasilkan perubahan konsistensi secara spasial, perubahan hubungan antar variabel, melawan aturan hukum konservasi energi, tidak memperhitungkan ketidakpastian dari kumpulan data observasi dan variabilitas internal, dan galat model dan fungsi koreksi umumnya dianggap stasioner terhadap waktu [@ehret2012hess; @addor2015; @maraun2016]. Ringkasan pengklasifikasian metode koreksi bias dapat dilihat pada [@tbl-watanabe] [@watanabe2012].

+---------------+--------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|               | Tipe konstan                                                                         | Tipe variabel                                                                                                                                      | Tipe parametrik                                                                                | Tipe non-parametrik                                                                                                                            |
+===============+======================================================================================+====================================================================================================================================================+================================================================================================+================================================================================================================================================+
| Definisi      | Nilai statistik pada periode masa depan tidak dimasukkan pada persamaan koreksi bias | Nilai statistik pada periode masa depan dimasukkan pada persamaan koreksi bias                                                                     | Distribusi parametrik diasumsikan masuk ke simulasi dan/atau data observasi                    | Distribusi parametrik tidak digunakan dalam koreksi bias                                                                                       |
+---------------+--------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
| Karakteristik | -   Statistik pada periode masa depan tidak mempengaruhi hasil dari koreksi bias     | -   Statistik pada periode masa depan mempengaruhi hasil dari koreksi bias                                                                         | -   Kuat secara statistik                                                                      | -   Diperlukan ekstrapolasi untuk nilai-nilai yang lebih/kurang dari periode dasar                                                             |
|               | -   Panjang data koreksi bias dapat diperoleh berapapun panjang periodenya           | -   Mudah untuk melakukan penyesuaian perubahan secara statistik dari periode dasar ke masa depan antara data belum terkoreksi dan data terkoreksi |                                                                                                |                                                                                                                                                |
|               |                                                                                      | -   Panjang data terkoreksi bias harus sama dengan data periode dasar                                                                              | -   Nilai yang tidak realistis dapat dihasilkan jika distribusi tidak diperkirakan dengan baik | -   Karena derajat bebas sama dengan jumlah parameter, ada permasalahan dalam *functional robustness* jika asumsinya distribusi non parametrik |
+---------------+--------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+

: Klasifikasi metode koreksi bias statistik {#tbl-watanabe tbl-colwidths="\[20, 25, 25, 30\]"}

Modul ini menjelaskan tata cara melakukan koreksi bias statistik dengan metode Delta dan Distribusi Statistik.

## Metode Delta

Cara paling sederhana untuk melakukan koreksi bias adalah dengan menambahkan/mengalikan data model ke data observasi pada periode dasar. @eq-1 umum digunakan pada koreksi data suhu dan @eq-2 untuk curah hujan.

$$
x_{cor, i} = x_{o,i} + \mu_{p} - \mu_{b}\tag{1}
$$ {#eq-1}

$$
x_{cor,i} = x_{o, i} . \frac{\mu_p}{\mu_b} \tag{2}
$$ {#eq-2}

dimana $x_{cor,i}$ dan $x_{o,i}$ $(i = 1, 2, …)$ secara berturut-turut dinotasikan sebagai data terkoreksi bias dan observasi dalam periode dasar tertentu, $\mu$ adalah rata-rata. Subskrip $b$, $o$, dan $p$ secara berturut-turut adalah data dasar, observasi, dan proyeksi

## Metode Distribusi Statistik

Koreksi bias dengan metode ini berdasarkan distribusi statistik dari data model yang ditransformasikan untuk menyesuaikan distribusi data dari observasi. Distribusi normal umum diasumsikan untuk data suhu udara [@piani2010suhu], sedangkan distribusi gamma untuk data curah hujan [@piani2010pr]. Persamaan *Probability Density Function* (PDF) untuk sebaran normal (@eq-3) dan sebaran gamma (@eq-4) adalah sebagai berikut.

$$ 
f(x, \mu, \sigma) = \frac{1}{\sigma \sqrt{2\pi}} ~ e^{-0.5 (\frac{x - \mu}{\sigma})^2} \tag{3}
$$ {#eq-3}

$$
f(x, \alpha, \beta) = \frac{e^{-\frac{x}{\beta}} ~ x^{\alpha-1}}{\Gamma(\alpha)\beta^\alpha} \tag{4}
$$ {#eq-4}

dengan x adalah data curah hujan atau suhu udara, $\alpha$ adalah parameter skala, $\beta$ adalah parameter bentuk, $\sigma$ adalah standar deviasi, dan $\mu$ adalah rata-rata. 

Di dalam publikasi Piani [@piani2010pr], persamaan yang digunakan untuk menghitung koreksi bias hujan harian dengan persamaan *Cumulative Density Function* (CDF) sebaran gamma ([@eq-8]), di mana $F(0)$ adalah peluang hari tidak hujan.

$$
F(x, \alpha, \beta) = 
  \begin{cases}
    \int_{0}^{x} \frac{e^{-\frac{x}{\beta}} ~ x^{\alpha-1}}{\Gamma(\alpha)\beta^\alpha} dx + F(0) &, ~ x > 0 \\
    F(0) &, ~ x = 0
  \end{cases} \tag{5}
$$ {#eq-5}

$$
F(0) = \frac{jumlah~hari~tidak~hujan}{jumlah~data} \tag{6}
$$ {#eq-6}

CDF ditentukan dari dua tipe data, yaitu observasi dan model. Bentuk *transfer function* untuk koreksi bias hujan harian diperoleh dari inverse CDF model dan observasi. Persamaannya adalah sebagai berikut

$$
F^{-1}(p, \alpha, \beta) = 
  \begin{cases}
    \frac{\frac{1}{\beta}^{\alpha}}{\Gamma(\alpha)} (p - F(0))^{-\alpha - 1} exp(-\frac{1}{\beta (p - cdf(0))}) &, ~ p > F(0) \\
    0 &, ~ p \leq F(0)
  \end{cases} \tag{7}
$$ {#eq-7}

dengan nilai $p$ adalah peluang ($ 0 < p \leq 1 $). Dengan mengetahui inverse CDF dari kedua data, hujan harian dari model terkoreksi dapat dihitung dengan persamaan regresi. Anda dapat menggunakan bentuk regresi seperti linier, kuadratik, atau eksponensial, tergantung dari hubungan antara inverse CDF model dan observasi. Untuk menghindari nilai negatif, nilai intersep pada persamaan regresi diatur menjadi 0.

Proses koreksi bias dihitung dari invers CDF. Pada @eq-4, nilai $\alpha$ dan $\beta$ dapat diestimasi dengan metode Momen dengan persamaan sebagai berikut 

$$
\alpha = \frac{n \bar{x}^2}{\Sigma_{i=1}^{n} (x_i - \bar{x})^2} \tag{8}
$$ {#eq-8}

$$
\beta = \frac{\Sigma_{i=1}^{n} (x_i - \bar{x}^2)}{n \bar{x}} \tag{9}
$$ {#eq-9}

Selain itu, dapat juga menggunakan *Maximum Likelihood Estimation* (MLE) dengan metode Greenwood dan Durand [@greenwood1960]. Persamaannya sebagai berikut.

$$
\alpha = 
  \begin{cases}
    \frac{0.5000876+0.1648852y−0.0544276y^2}{y} &,~ \text{jika} ~ 0 \leq y \leq 0.5772 \\
    \frac{8.898919+9.05995y+0.9775373y^2}{17.79728y+11.968477y^2+y^3} &, ~ \text{jika} ~ 0.5772 < y \leq 17 \\
    \frac{1}{y} &, ~ \text{jika} ~ y > 17
  \end{cases} \tag{10}
$$ {#eq-10}

$$
y = ln(\bar{x}) - \frac{1}{n} \sum_{i = 1} ^ n ln(x_i) \tag{11}
$$ {#eq-11}

$$
\beta = \frac{\bar{x}}{\alpha} \tag{12}
$$ {#eq-12}

Pada modul ini, penentuan parameter pada distribusi gamma menggunakan metode MLE. 

## Sebelum Mulai

Variabel yang akan dikoreksi adalah curah hujan harian dari data Model [CNRM-CM6-1](https://www.umr-cnrm.fr/cmip6) untuk *Baseline* dan *Projection* SSP245 terhadap data [Climate Hazards group InfraRed Precipitation with Stations (CHIRPS)](https://data.chc.ucsb.edu/products/). Model CNRM-CM6-1 dapat Anda unduh pada halaman [Climate Data Store (CDS)](https://cds.climate.copernicus.eu) dan Anda dapat memilih lokasi. Data contoh yang telah kami unduh sudah dipilih berdasarkan wilayah Indonesia. Format data berbentuk netCDF (`.nc`). [@tbl-data] adalah informasi penggunaan data pada modul ini.

| Data       | Tipe             | Periode     | Resolusi      | Sumber                                                                   |
| ---------- | ---------------- | ----------- | ------------- | ------------------------------------------------------------------------ |
| CHIRPS     | Historis         | 1981 - 2020 | 0.05° x 0.05° | [https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p05/] |
| CNRM-CM6-1 | Historis         | 1950 - 2014 | 1.41° x 1.41° | [https://cds.climate.copernicus.eu/]                                     |
| CNRM-CM6-1 | Proyeksi SSP 245 | 2021 - 2100 | 1.41° x 1.41° | [https://cds.climate.copernicus.eu/]                                     |

: Informasi data yang digunakan pada modul ini {#tbl-data}

Satuan curah hujan pada data CHIRPS berbeda dengan CNRM-CM6-1. Satuan curah hujan CHIRPS adalah milimeter per hari, sedangkan CNRM-CM6-1 adalah kg/m²/s. Untuk mengubah satuan pada model CNRM-CM6-1, Anda dapat menggunakan [@eq-13].

$$
CH (mm/hari) = CH (kg/m^2/s) \times 86400 \tag{13}
$$ {#eq-13}

## Pengolahan Data

Koreksi bias curah hujan harian menggunakan data spasial. Teknik koreksi bias dengan Metode Delta dan Distribusi dijelaskan pada subbab ini menggunakan R, Python, dan Julia.

### 1. R {.unnumbered}

Sebelum menuju ke metode koreksi bias, data-data yang telah kami sediakan perlu dilakukan penyesuaian/modifikasi terlebih dahulu. Hal ini dikarenakan cakupan area dan resolusi spasial maupun periode dari masing-masing data berbeda. Untuk itu, kami akan melakukan beberapa tahapan berikut ini.

1. Impor terlebih dahulu paket yang digunakan. Jika belum ada, silahkan pasang terlebih dahulu dengan fungsi `install.packages()`.
   ```r
   library(dplyr)
   library(ggplot2)
   library(glue)
   library(ncdf4)
   library(terra)
   ```
2. Bacalah ketiga data dengan fungsi `rast()`. 
   ```r
   # Direktori/folder data
   dpth <- 'data/' 
   # Membaca data
   chp <- rast(glue('{dpth}/chirps-v2.0.1981-2020.days_p05.nc'))
   bsl <- rast(glue('{dpth}/pr_day_CNRM-CM6-1_historical_r1i1p1f2_gr_19500101-20141231_v20180917.nc'))
   ssp <- rast(glue('{dpth}/pr_day_CNRM-CM6-1_ssp245_r1i1p1f2_gr_20150101-21001231_v20190219.nc'))
   ```
3. Tentukan cakupan area serta waktu untuk koreksi. Misalnya, lokasi yang dipilih adalah Provinsi Jawa Timur dengan periode dari tahun 1981-2010 untuk baseline dan tahun 2031-2060 untuk skenario.
   ```r
   # Cakupan area
   kalimantan <- ext(108, 123, -4, 8)
   chp <- crop(chp, kalimantan)
   bsl <- crop(bsl, kalimantan)
   ssp <- crop(ssp, kalimantan)
   # Periode koreksi
   his_time <- seq(as.Date('1981-01-01'), as.Date('2010-12-31'), by = 'day')
   prj_time <- seq(as.Date('2031-01-01'), as.Date('2060-12-31'), by = 'day')
   
   chp <- raster::subset(chp, which(getZ(chp) %in% his_time))
   bsl <- raster::subset(bsl, which(getZ(bsl) %in% his_time))
   ssp <- raster::subset(prj, which(getZ(prj) %in% prj_time))
   ```  
4. Samakan resolusi spasial CNRM-CM6-1 dengan CHIRPS dengan metode interpolasi. Secara default, metode interpolasi yang digunakan adalah *bilinear*. Anda dapat mengganti metodenya dengan menambahkan argumen `method` di dalam fungsi `resample()`. Metode yang tersedia adalah *nearest neighbor*, *cubicspline*, *lanczos*, atau *sum*.
   ```r
   bsl <- resample(bsl, chp)
   ssp <- resample(ssp, chp)
   ``` 
5. Selanjutnya, konversikan satuan curah hujan CNRM-CM6-1 menjadi mm/hari dengan [@eq-13]. Setelah itu, nilai hasil konversi diubah menjadi 0 mm/hari jika nilainya kurang dari 1 mm/hari.
   ```r
   bsl <- bsl * 86400
   ssp <- ssp * 86400

   bsl[bsl < 1] <- 0
   ssp[ssp < 1] <- 0
   ```

#### Metode Delta {#sec-rmetodedelta .unnumbered}

1. Dengan menggunakan [@eq-2], carilah rata-rata curah hujan dari data CNRM-CM6-1 untuk proyeksi dan baseline.
   ```r
   bsl_mean <- calc(bsl, mean)
   ssp_mean <- calc(ssp, mean)
   ```
2. Carilah rasio antara rata-rata baseline dengan proyeksi.
   ```r
   ratio <- ssp_mean / bsl_mean
   ```
3. Terakhir, kalikan rasio dengan CHIRPS sehingga diperoleh curah hujan proyeksi terkoreksi.
   ```r
   ssp_corr <- ratio * chirps
   ```
4. 

#### Metode Distribusi {#sec-rmetodedist .unnumbered}

1. aaaa
2. 2222
3. 2
4. 2
5. 2
6. 2
7. 2
8. 2
9. 2
10. 

### 2. Python {.unnumbered}

Beberapa paket Python yang digunakan adalah `cartopy`, `matplotlib`, dan `xarray`. Jika Anda menggunakan `conda`, pasang terlebih dahulu paket-paket tersebut, ditambah dengan paket `jupyter`, `netcdf4` dan `dask` dengan membuat *environment* baru, misalnya `bias-correction`. Perintahnya diketik di terminal Bash (Linux/Mac) atau Anaconda3 Prompt (Windows) sebagai berikut.

```bash
conda create -n bias-correction -c conda-forge xarray matplotlib cartopy jupyter netcdf4 dask
```

Untuk menuliskan skrip Python, ketik `jupyter notebook` pada terminal. Kemudian, muncul tampilan Jupyter Notebook pada Browser default Anda dan Anda dapat langsung membuat skrip dengan menekan tombol ***New -> Python 3 (ipykernel)***.

1. Impor paket-paket yang telah terpasang
   ```python
   import numpy as np
   import pandas as pd
   import xarray as xr
   import matplotlib.pyplot as plt
   import cartopy.crs as ccrs
   ```
2. Impor ketiga data dengan menggunakan `xarray`. 
   ```python
   pth = 'data'
   chp = xr.open_dataset(f'{pth}/chirps-v2.0.1981-2020.days_p05.nc', chunks={'time': 30})
   bsl = xr.open_dataset(f'{pth}/pr_day_CNRM-CM6-1_historical_r1i1p1f2_gr_19500101-20141231_v20180917.nc', chunks={'time': 30})
   ssp = xr.open_dataset(f'{pth}/pr_day_CNRM-CM6-1_ssp245_r1i1p1f2_gr_20150101-21001231_v20190219.nc', chunks={'time': 30})
   ```
   Argumen `chunks` digunakan untuk membagi data menjadi beberapa bagian yang lebih kecil. Hal ini dilakukan untuk mengurangi penggunaan memori. `chunks={'time': 30}` berarti data dibagi menjadi 30 bagian. Argumen ini dapat diaktifkan apabila Anda telah memasang `dask`.
3. Ubah nama dari koordinat `longitude` dan `latitude` menjadi `lon` dan `lat` pada data CHIRPS, juga variabel curah hujan `precip` menjadi `pr`. Langkah ini dilakukan agar nama koordinat serta variabel sama dengan data baseline maupun skenario. Selain itu, ini dapat memudahkan Anda dalam menghitung ketika melakukan koreksi bias. 
   ```python
   chp = chp.rename({'longitude' : 'lon', 'latitude' : 'lat', 'precip' : 'pr'})
   ```
4. Tentukan cakupan area serta waktu untuk koreksi. Misalnya, lokasi yang dipilih adalah Pulau Kalimantan dengan periode dari tahun 1981-2010 untuk *Baseline* dan tahun 2031-2060 untuk skenario SSP 245.
   ```python
   # Cakupan area
   lon1, lon2, lat1, lat2 = 108, 123, -4, 8

   # Potong data sesuai cakupan area
   chp = chp.sel(lon=slice(lon1, lon2), lat=slice(lat1, lat2))
   bsl = bsl.sel(lon=slice(lon1, lon2), lat=slice(lat1, lat2))
   ssp = ssp.sel(lon=slice(lon1, lon2), lat=slice(lat1, lat2))

   # Periode waktu untuk koreksi
   t1, t2 = '1981-01-01', '2010-12-31'
   t3, t4 = '2031-01-01', '2060-12-31'

   # Potong data sesuai periode waktu
   chp = chp.sel(time=slice(t1, t2))
   bsl = bsl.sel(time=slice(t1, t2))
   ssp = ssp.sel(time=slice(t3, t4))
   ```
5. Samakan resolusi spasial CNRM-CM6-1 terhadap CHIRPS dengan metode interpolasi linier. Anda dapat memilih metode interpolasi lain, seperti `nearest`, `zero`, `slinear`, `quadratic`, `cubic`, atau `polynomial`. Kegunaan dari mencantumkan `kwargs` adalah untuk mengatasi nilai `NaN` yang muncul akibat interpolasi. Nilai `NaN` tersebut diubah menjadi nilai hasil ekstrapolasi.
   ```python
   # Interpolasi CNRM-CM6-1 ke resolusi CHIRPS
   resample_bsl = bsl.interp(coords = {
      'lat' : chp['latitude'],
      'lon' : chp['longitude']
    }, method = 'linear', kwargs={"fill_value": "extrapolate"})
    
   resample_ssp = ssp.interp(coords = {
      'lat' : chp['latitude'],
      'lon' : chp['longitude']
   }, method = 'linear', kwargs={"fill_value": "extrapolate"})
   ```
6. Selanjutnya, konversikan satuan curah hujan CNRM-CM6-1 menjadi mm/hari dengan [@eq-13]. Setelah itu, nilai hasil konversi diubah menjadi 0 mm/hari jika nilainya <1 mm/hari.
   ```python
   # Baseline
   resample_bsl['pr'] = resample_bsl['pr'] * 86400
   resample_bsl['pr'] = xr.where(resample_bsl['pr'] < 1, 0, resample_bsl['pr'])
   # SSP 245
   resample_ssp['pr'] = resample_ssp['pr'] * 86400
   resample_ssp['pr'] = xr.where(resample_ssp['pr'] < 1, 0, resample_ssp['pr'])
   ```

Setelah mengerjakan langkah-langkah di atas, Anda dapat melanjutkan langkah-langkah koreksi bias pada subbab berikutnya.

#### Metode Delta {#sec-pymetodedelta .unnumbered}

1. Dengan menggunakan [@eq-2], carilah rata-rata curah hujan dari data CNRM-CM6-1 untuk proyeksi dan baseline untuk semua waktu.
   ```python
   # Baseline
   bsl_mean = resample_bsl.mean(dim='time')
   # SSP 245
   ssp_mean = resample_ssp.mean(dim='time')
   ```
2. Hitung rasio antara rata-rata baseline dengan proyeksi.
   ```python
   ratio = ssp_mean / bsl_mean
   ```
3. Terakhir, kalikan rasio dengan CHIRPS sehingga diperoleh curah hujan proyeksi terkoreksi.
   ```python
   # Kalikan rasio dengan data CHIRPS
   ssp_corrected = ratio * chp

   # Lakukan transpos dimensi agar dimensi waktu berada di posisi pertama
   ssp_corrected = ssp_corrected.transpose('time', 'lat', 'lon')

   # Mengubah waktu historis menjadi skenario SSP
   ssp_corrected = ssp_corrected.assign(time=ssp.time)
   ```

Langkah 1-3 merupakan proses perhitungan koreksi bias Metode Delta. Selanjutnya, kita akan memvisualisasikan hasil koreksi bias tersebut. Sebagai contoh, kita akan memvisualisasikan rataan klimatologi bulanan (2021-2050) hasil koreksi bias secara spasial, serta perbandingan data CNRM-CM6-1 SSP 245 sebelum dan sesudah terkoreksi.

1. Lakukan perhitungan akumulasi curah hujan bulanan
   ```python
   # Akumulasi curah hujan bulanan
   ssp_corrected_monthly = ssp_corrected.resample(time='1M').sum(skipna=False)
   ```
2. Lakukan perhitungan rata-rata klimatologi bulanan
   ```python
   # Rata-rata klimatologi bulanan
   ssp_corrected_monthly_mean = ssp_corrected_monthly.groupby('time.month').mean(dim='time')
   ```
3. Visualisasikan hasil koreksi bias secara spasial
   ```python
   # Create a subplot of spatial plots for each month
   fig, axes = plt.subplots(3, 4, figsize = (15, 10), subplot_kw = {'projection': ccrs.PlateCarree()})
   for month, ax in zip(range(1, 13), axes.ravel()):
      # Memilih data untuk setiap bulan
      month_data = ssp_corrected_monthly_mean.sel(month=month)
      month_data = month_data['pr']

      # Membuat plot
      month_data.plot.imshow(ax=ax, cmap='jet', add_colorbar=False, transform=ccrs.PlateCarree())

      # Menambahkan judul grafik dan garis pantai
      ax.set_title(f'Month: {month}')
      ax.coastlines(linewidth=0.5)

      # Menambahkan garis lintang dan bujur
      gl = ax.gridlines(draw_labels=True, linestyle='--', linewidth=0.5)

      # Menghilangkan label untuk beberapa plot agar tidak terlalu padat
      if month not in [1, 5, 9]:
           gl.left_labels = False
      if month not in [9, 10, 11, 12]:
           gl.bottom_labels = False
      if month not in [4, 8, 12]:
           gl.right_labels = False
      gl.top_labels = False
   
   # Menambahkan colorbar
   cbar_ax = fig.add_axes([0.15, -0.05, 0.7, 0.05])
   fig.colorbar(ax.images[0], cax=cbar_ax, orientation='horizontal', label='Curah hujan (mm/bulan)')
   
   plt.show()
   ```
   ![Koreksi bias metode delta rataan bulanan klimatologi (2021-2050)](./Pictures/py-sp-delta-correction.png)
4. Selanjutnya, buat grafik garis untuk membandingkan data CNRM-CM6-1 SSP 245 sebelum dan sesudah terkoreksi. Sebagai contoh, lokasi Kota Banjarmasin digunakan sebagai perbandingan.
   ```python
   # Membuat plot
   fig, ax = plt.subplots(figsize=(10, 5))

   # Memilih data untuk setiap bulan
   lat, lon = -3.322423232458966, 114.59597874153853
   ssp_corrected.pr.sel(lat=lat, lon=lon, method='nearest').plot(ax=ax, label='Corrected')
   ssp_mean.pr.sel(lat=lat, lon=lon, method='nearest').plot(ax=ax, label='Raw')

   # Menambahkan judul grafik
   ax.set_title('Perbandingan data CNRM-CM6-1 SSP 245 sebelum dan sesudah terkoreksi')

   # Menambahkan label sumbu x dan y
   ax.set_xlabel('Tanggal')
   ax.set_ylabel('Curah hujan (mm/hari)')

   # Menambahkan legenda
   ax.legend()
   plt.show()
   ```
   ![Perbandingan data CNRM-CM6-1 SSP 245 sebelum dan sesudah terkoreksi](./Pictures/py-ts-delta-correction.png)

#### Metode Distribusi {#sec-pymetodedist .unnumbered}